## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/135bert-base-last/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_1_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_2_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_3_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_4_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_5_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.902665,0.562238,0.285580,0.407876,0.641859,0.489384,0.654044,0.625419,0.709666,...,0.890224,0.859752,0.527212,0.935704,0.922786,0.735822,0.081236,0.057967,0.780899,0.875334
1,46,0.893521,0.567713,0.003560,0.827061,0.818361,0.950422,0.597164,0.506655,0.040890,...,0.715118,0.956614,0.665624,0.976948,0.984512,0.879773,0.908772,0.064430,0.067630,0.894909
2,70,0.918040,0.655061,0.017824,0.765053,0.910708,0.949870,0.593809,0.528530,0.214804,...,0.848186,0.942204,0.619218,0.974075,0.974692,0.854745,0.123994,0.087013,0.876335,0.903425
3,132,0.892383,0.424284,0.005144,0.705620,0.800668,0.902523,0.528269,0.436055,0.102040,...,0.741364,0.945694,0.664074,0.975808,0.982650,0.894760,0.843216,0.173146,0.403764,0.902113
4,200,0.907307,0.407312,0.041510,0.807743,0.773737,0.844129,0.635591,0.606069,0.108127,...,0.692439,0.899892,0.616416,0.954220,0.944209,0.802718,0.200546,0.124758,0.553059,0.886336


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.889733,0.593635,0.037380,0.697889,0.830637,0.844961,0.579766,0.503115,0.238034,...,0.814494,0.924260,0.650576,0.962710,0.966617,0.852927,0.502910,0.135664,0.491251,0.899979
std,2812.670060,0.045582,0.125616,0.073296,0.120767,0.098108,0.131180,0.051167,0.087896,0.196825,...,0.076670,0.029496,0.046935,0.013840,0.015633,0.045141,0.318309,0.068407,0.278347,0.021473
min,39.000000,0.753243,0.350955,0.001453,0.201745,0.379421,0.205984,0.450284,0.349453,0.002226,...,0.582098,0.818061,0.527212,0.917060,0.888192,0.713262,0.003476,0.004652,0.019203,0.813881
25%,2572.000000,0.858083,0.486888,0.006768,0.629179,0.781904,0.817216,0.542779,0.439121,0.082525,...,0.754073,0.907792,0.616520,0.954221,0.958615,0.824953,0.163929,0.090228,0.246598,0.886851
50%,5093.000000,0.890614,0.587173,0.011209,0.699703,0.842159,0.893704,0.573129,0.478800,0.167291,...,0.817175,0.929303,0.650409,0.964560,0.970069,0.859571,0.562834,0.132310,0.467795,0.901669
75%,7482.000000,0.924146,0.688982,0.027071,0.777887,0.900725,0.926375,0.611564,0.548291,0.355658,...,0.879805,0.945852,0.683300,0.972794,0.977728,0.885885,0.801379,0.177637,0.736886,0.914316
max,9640.000000,0.982324,0.924159,0.577758,0.972446,0.985697,0.979056,0.723861,0.800611,0.862756,...,0.965336,0.983934,0.822849,0.989376,0.992825,0.969227,0.967470,0.359208,0.991882,0.955422
